### Import Libraries

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

### Highlight the detected shape of the template on the image

In [2]:
def highlight_shape(img, temp_img, position):
    result = img.copy()
    result[position[1]: position[1] + temp_img.shape[0],
           position[0]: position[0] + tesmp_img.shape[1]][temp_img != 0] = [255, 0, 0]
    return result

### Read template image and video

In [6]:
temp = cv2.imread('hand_template.bmp', 0)
height, width = temp.shape[::]
cap = cv2.VideoCapture('test2.wmv')
frame_exists, frame = cap.read()

### For each frame:
* Convert the frame to gray scale
* Apply Canny edge detection
* Apply distance transformation
* Get the chamfer matching by correlating the transformed frame with the template
* Get the location of the maximum matched value
* Highlight the shape at that location

In [8]:
while frame_exists:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 1)
    edges = cv2.Canny(gray, 50, 200)
    dist = cv2.distanceTransform(edges, cv2.DIST_L2, 3)
    cv2.normalize(dist, dist, 0, 255, cv2.NORM_MINMAX)
    dist = dist.astype(np.uint8)
    cm = (cv2.matchTemplate(dist, temp, cv2.TM_CCORR) / temp.sum()).round()
    cm = cm.astype(np.uint8)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(cm)
    top_left = max_loc
    hl = highlight_shape(frame, temp, top_left)
    cv2.imshow('cm', cm)
    cv2.imshow('Edges', edges)
    cv2.imshow('Tracking', hl)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    frame_exists, frame = cap.read()

cap.release()
cv2.destroyAllWindows()